In [1]:
import pandas as pd
import matplotlib
%matplotlib inline
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error
import pickle
#from  models.populate_dbbikes_database import *
import matplotlib.dates as mdates


In [2]:
from sqlalchemy import create_engine
USER = "tim"
PASSWORD = "Ylu3shin123!!"
URI = "dbbikes.cjlp5umzs5hs.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

In [3]:
# USER = "ciara"
# PASSWORD = "Ciara123"
# PORT = "3306"
# DB = "dbbikes"
# URI = "dbbikes.cvr6gofoxmkp.us-east-2.rds.amazonaws.com"
# engine = create_engine("mysql+pymysql://{}:{}@{}:{}/{}".format(USER, PASSWORD, URI, PORT, DB), echo=True)

### Bike Availability

In [4]:
sql = f"""
SELECT id, number, last_update, available_bikes, available_bike_stands FROM stations
GROUP BY id, day(last_update)
ORDER by id, last_update ASC;
"""
#last_update=datetime.datetime.fromtimestamp(station.get('last_update') / 1e3)
#print(sql)

In [5]:
bike_df = pd.read_sql_query(sql, engine)

2021-04-09 18:57:37,241 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'sql_mode'
2021-04-09 18:57:37,243 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-09 18:57:37,338 INFO sqlalchemy.engine.Engine SHOW VARIABLES LIKE 'lower_case_table_names'
2021-04-09 18:57:37,339 INFO sqlalchemy.engine.Engine [generated in 0.00068s] {}
2021-04-09 18:57:37,526 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2021-04-09 18:57:37,527 INFO sqlalchemy.engine.Engine [raw sql] {}
2021-04-09 18:57:37,809 INFO sqlalchemy.engine.Engine 
SELECT id, number, last_update, available_bikes, available_bike_stands FROM stations
GROUP BY id, day(last_update)
ORDER by id, last_update ASC;

2021-04-09 18:57:37,810 INFO sqlalchemy.engine.Engine [raw sql] {}


In [6]:
bike_df['last_update'] = bike_df['last_update'].astype('datetime64[ns]')

#from datetime import datetime
bike_df['last_update'] = pd.to_datetime(bike_df['last_update'])

#bike_df.last_update = datetime.utcfromtimestamp(bike_df.last_update).strftime('%Y-%m-%d %H:%M:%S')
bike_df  

id  number         last_update  available_bikes  \
0            1      42 2021-03-16 19:32:50               23   
1            2      30 2021-03-16 19:26:54                4   
2            3      54 2021-03-16 19:28:27                3   
3            4     108 2021-03-16 19:29:25               23   
4            5      56 2021-03-16 19:34:39                6   
...        ...     ...                 ...              ...   
693619  693620      39 2021-04-09 17:49:50               12   
693620  693621      83 2021-04-09 17:50:21               22   
693621  693622      92 2021-04-09 17:52:18                8   
693622  693623      21 2021-04-09 17:53:39                8   
693623  693624      88 2021-04-09 17:47:47                9   

        available_bike_stands  
0                           7  
1                          16  
2                          30  
3                          17  
4                          34  
...                       ...  
693619                      8  
693620                     17  
693621                     32  
693622                     22  
693623                     21  

[693624 rows x 5 columns]

In [7]:
bike_df.dtypes

id                                int64
number                            int64
last_update              datetime64[ns]
available_bikes                   int64
available_bike_stands             int64
dtype: object

In [8]:
bike_df.shape

(693624, 5)

In [9]:
bike_df.tail()

id  number         last_update  available_bikes  \
693619  693620      39 2021-04-09 17:49:50               12   
693620  693621      83 2021-04-09 17:50:21               22   
693621  693622      92 2021-04-09 17:52:18                8   
693622  693623      21 2021-04-09 17:53:39                8   
693623  693624      88 2021-04-09 17:47:47                9   

        available_bike_stands  
693619                      8  
693620                     17  
693621                     32  
693622                     22  
693623                     21

### Weather

In [10]:
sql = f"""

SELECT * FROM weather

"""


In [11]:
weather_df = pd.read_sql_query(sql, engine)

2021-04-09 18:58:19,802 INFO sqlalchemy.engine.Engine 

SELECT * FROM weather


2021-04-09 18:58:19,803 INFO sqlalchemy.engine.Engine [raw sql] {}


In [12]:
weather_df.dtypes

id                              int64
description                    object
temp                          float64
last_update_weather    datetime64[ns]
dtype: object

In [13]:
# category    
weather_df['description'] = weather_df['description'].astype('category')    

# datetime   
weather_df['last_update_weather'] = weather_df['last_update_weather'].astype('datetime64[ns]')

In [14]:
weather_df.dtypes

id                              int64
description                  category
temp                          float64
last_update_weather    datetime64[ns]
dtype: object

In [15]:
weather_df.shape

(2874, 4)

In [16]:
weather_df.tail()

id    description  temp last_update_weather
2869  2870  broken clouds  7.50 2021-04-07 13:34:08
2870  2871  broken clouds  7.50 2021-04-07 13:40:02
2871  2872  broken clouds  7.41 2021-04-07 13:53:58
2872  2873  broken clouds  7.60 2021-04-07 14:04:43
2873  2874  broken clouds  7.88 2021-04-07 14:16:41

### Together - using pandas merge_asof

In [17]:
# https://pandas.pydata.org/docs/reference/api/pandas.merge_asof.html

# closest to 

In [18]:
bike_df.sort_values('last_update', inplace=True)
weather_df.sort_values('last_update_weather', inplace=True) 

In [19]:
df = pd.merge_asof(bike_df, weather_df, left_on='last_update', right_on='last_update_weather')

In [20]:
df.shape

(693624, 9)

In [21]:
df = df.drop(['last_update_weather'], axis=1)

In [22]:
df.tail(5)

id_x  number         last_update  available_bikes  \
693619  693556      73 2021-04-09 17:54:18                7   
693620  693537      90 2021-04-09 17:54:20                7   
693621  693548      67 2021-04-09 17:54:22               19   
693622  693559      19 2021-04-09 17:54:27               14   
693623  693516      42 2021-04-09 17:54:31               27   

        available_bike_stands    id_y    description  temp  
693619                     23  2874.0  broken clouds  7.88  
693620                     33  2874.0  broken clouds  7.88  
693621                     21  2874.0  broken clouds  7.88  
693622                     16  2874.0  broken clouds  7.88  
693623                      3  2874.0  broken clouds  7.88

In [23]:
df = df.drop(['id_y'], axis=1)

In [24]:
df = df.drop(['id_x'], axis=1)

In [25]:
df.tail(5)

number         last_update  available_bikes  available_bike_stands  \
693619      73 2021-04-09 17:54:18                7                     23   
693620      90 2021-04-09 17:54:20                7                     33   
693621      67 2021-04-09 17:54:22               19                     21   
693622      19 2021-04-09 17:54:27               14                     16   
693623      42 2021-04-09 17:54:31               27                      3   

          description  temp  
693619  broken clouds  7.88  
693620  broken clouds  7.88  
693621  broken clouds  7.88  
693622  broken clouds  7.88  
693623  broken clouds  7.88

In [26]:
df.shape

(693624, 6)

In [27]:
print('Number of duplicate (excluding first) rows in the table is: ', df.duplicated().sum())

print('Number of duplicate rows (including first) in the table is:', df[df.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  315088
Number of duplicate rows (including first) in the table is: 622960


In [28]:
df = df.drop_duplicates(keep='last')
# get duplicated row data. Use "keep=False" to mark all duplicates as true
print('Duplicate remaining rows:', df[df.duplicated(keep='last')].shape[0])

Duplicate remaining rows: 0


In [29]:
df.shape

(378536, 6)

### The stations

In [30]:
#stations = list(sorted(df.number.unique()))
#print(stations)
#print(len(stations))

### The days

### The hours

In [31]:
# New column called hourly 
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.hour.html
df['hourly'] = df['last_update'].dt.hour

# https://stackoverflow.com/questions/61124647/pandas-how-to-get-dummies-on-time-series-data
#  hours 0 to 23 will be 0 with one hour being 1
#hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')

hourly_dummies = pd.get_dummies(df['hourly'], prefix='hour')

# https://towardsdatascience.com/pandas-concat-tricks-you-should-know-to-speed-up-your-data-analysis-cd3d4fdfe6dd
df = pd.concat([df, hourly_dummies], axis=1)

# then drop original column - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html


# list of  hour dummies- # https://datatofish.com/list-column-names-pandas-dataframe/
hour_dummies = list(hourly_dummies)

df = df.drop(['hourly'], axis=1)


### The dataframes

In [32]:
# New column called weekday with the number of the day of the week
df['weekday'] = df['last_update'].dt.dayofweek 
# https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.weekday.html
df_monday = df[df.weekday == 0]
df_tuesday = df[df.weekday == 1]
df_wednesday = df[df.weekday == 2]
df_thursday = df[df.weekday == 3]
df_friday = df[df.weekday == 4]
df_saturday = df[df.weekday == 5]
df_sunday = df[df.weekday == 6]


df_monday

number         last_update  available_bikes  available_bike_stands  \
104698      30 2021-03-22 00:00:03                0                     20   
104700      45 2021-03-22 00:00:05                0                     30   
104702      57 2021-03-22 00:00:06                4                     19   
104704     115 2021-03-22 00:00:07               30                      0   
104706      87 2021-03-22 00:00:20               11                     27   
...        ...                 ...              ...                    ...   
576008      64 2021-04-05 23:59:09                1                     39   
576009      31 2021-04-05 23:59:16               20                      0   
576010      23 2021-04-05 23:59:29                2                     28   
576011      83 2021-04-05 23:59:32               22                     17   
576012     116 2021-04-05 23:59:36               13                     17   

          description  temp  hour_0  hour_1  hour_2  hour_3  ...  hour_15  \
104698  broken clouds  8.03       1       0       0       0  ...        0   
104700  broken clouds  8.03       1       0       0       0  ...        0   
104702  broken clouds  8.03       1       0       0       0  ...        0   
104704  broken clouds  8.03       1       0       0       0  ...        0   
104706  broken clouds  8.03       1       0       0       0  ...        0   
...               ...   ...     ...     ...     ...     ...  ...      ...   
576008      clear sky -0.15       0       0       0       0  ...        0   
576009      clear sky -0.15       0       0       0       0  ...        0   
576010      clear sky -0.15       0       0       0       0  ...        0   
576011      clear sky -0.15       0       0       0       0  ...        0   
576012      clear sky -0.15       0       0       0       0  ...        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
104698        0        0        0        0        0        0        0   
104700        0        0        0        0        0        0        0   
104702        0        0        0        0        0        0        0   
104704        0        0        0        0        0        0        0   
104706        0        0        0        0        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
576008        0        0        0        0        0        0        0   
576009        0        0        0        0        0        0        0   
576010        0        0        0        0        0        0        0   
576011        0        0        0        0        0        0        0   
576012        0        0        0        0        0        0        0   

        hour_23  weekday  
104698        0        0  
104700        0        0  
104702        0        0  
104704        0        0  
104706        0        0  
...         ...      ...  
576008        1        0  
576009        1        0  
576010        1        0  
576011        1        0  
576012        1        0  

[50406 rows x 31 columns]

In [33]:
df_monday

number         last_update  available_bikes  available_bike_stands  \
104698      30 2021-03-22 00:00:03                0                     20   
104700      45 2021-03-22 00:00:05                0                     30   
104702      57 2021-03-22 00:00:06                4                     19   
104704     115 2021-03-22 00:00:07               30                      0   
104706      87 2021-03-22 00:00:20               11                     27   
...        ...                 ...              ...                    ...   
576008      64 2021-04-05 23:59:09                1                     39   
576009      31 2021-04-05 23:59:16               20                      0   
576010      23 2021-04-05 23:59:29                2                     28   
576011      83 2021-04-05 23:59:32               22                     17   
576012     116 2021-04-05 23:59:36               13                     17   

          description  temp  hour_0  hour_1  hour_2  hour_3  ...  hour_15  \
104698  broken clouds  8.03       1       0       0       0  ...        0   
104700  broken clouds  8.03       1       0       0       0  ...        0   
104702  broken clouds  8.03       1       0       0       0  ...        0   
104704  broken clouds  8.03       1       0       0       0  ...        0   
104706  broken clouds  8.03       1       0       0       0  ...        0   
...               ...   ...     ...     ...     ...     ...  ...      ...   
576008      clear sky -0.15       0       0       0       0  ...        0   
576009      clear sky -0.15       0       0       0       0  ...        0   
576010      clear sky -0.15       0       0       0       0  ...        0   
576011      clear sky -0.15       0       0       0       0  ...        0   
576012      clear sky -0.15       0       0       0       0  ...        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
104698        0        0        0        0        0        0        0   
104700        0        0        0        0        0        0        0   
104702        0        0        0        0        0        0        0   
104704        0        0        0        0        0        0        0   
104706        0        0        0        0        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
576008        0        0        0        0        0        0        0   
576009        0        0        0        0        0        0        0   
576010        0        0        0        0        0        0        0   
576011        0        0        0        0        0        0        0   
576012        0        0        0        0        0        0        0   

        hour_23  weekday  
104698        0        0  
104700        0        0  
104702        0        0  
104704        0        0  
104706        0        0  
...         ...      ...  
576008        1        0  
576009        1        0  
576010        1        0  
576011        1        0  
576012        1        0  

[50406 rows x 31 columns]

In [34]:
df_monday.shape

(50406, 31)

In [35]:
df_tuesday

number         last_update  available_bikes  available_bike_stands  \
0           76 2021-03-16 19:24:43               10                     28   
1          110 2021-03-16 19:24:44               13                     27   
2           48 2021-03-16 19:24:45               12                     28   
3           79 2021-03-16 19:24:45                4                     23   
4           43 2021-03-16 19:24:52               13                     17   
...        ...                 ...              ...                    ...   
607400      78 2021-04-06 23:58:52               16                     24   
607401      83 2021-04-06 23:58:53               18                     21   
607402      62 2021-04-06 23:59:20                0                     40   
607403     116 2021-04-06 23:59:20                7                     23   
607404      75 2021-04-06 23:59:25               24                     16   

            description  temp  hour_0  hour_1  hour_2  hour_3  ...  hour_15  \
0                   NaN   NaN       0       0       0       0  ...        0   
1                   NaN   NaN       0       0       0       0  ...        0   
2                   NaN   NaN       0       0       0       0  ...        0   
3                   NaN   NaN       0       0       0       0  ...        0   
4                   NaN   NaN       0       0       0       0  ...        0   
...                 ...   ...     ...     ...     ...     ...  ...      ...   
607400  overcast clouds  2.46       0       0       0       0  ...        0   
607401  overcast clouds  2.46       0       0       0       0  ...        0   
607402  overcast clouds  2.46       0       0       0       0  ...        0   
607403  overcast clouds  2.46       0       0       0       0  ...        0   
607404  overcast clouds  2.46       0       0       0       0  ...        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
0             0        0        0        1        0        0        0   
1             0        0        0        1        0        0        0   
2             0        0        0        1        0        0        0   
3             0        0        0        1        0        0        0   
4             0        0        0        1        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
607400        0        0        0        0        0        0        0   
607401        0        0        0        0        0        0        0   
607402        0        0        0        0        0        0        0   
607403        0        0        0        0        0        0        0   
607404        0        0        0        0        0        0        0   

        hour_23  weekday  
0             0        1  
1             0        1  
2             0        1  
3             0        1  
4             0        1  
...         ...      ...  
607400        1        1  
607401        1        1  
607402        1        1  
607403        1        1  
607404        1        1  

[51999 rows x 31 columns]

In [36]:
df_sunday

number         last_update  available_bikes  available_bike_stands  \
73300       36 2021-03-21 00:00:00                9                     31   
73302        3 2021-03-21 00:00:04               20                      0   
73304       92 2021-03-21 00:00:09               21                     19   
73305       72 2021-03-21 00:00:09                6                     25   
73307       82 2021-03-21 00:00:15                6                     16   
...        ...                 ...              ...                    ...   
544725      69 2021-04-04 23:59:26               37                      2   
544726      23 2021-04-04 23:59:26                9                     21   
544727      73 2021-04-04 23:59:27                9                     21   
544728       3 2021-04-04 23:59:35                2                     18   
544729      99 2021-04-04 23:59:36                8                     22   

          description  temp  hour_0  hour_1  hour_2  hour_3  ...  hour_15  \
73300   broken clouds  8.52       1       0       0       0  ...        0   
73302   broken clouds  8.52       1       0       0       0  ...        0   
73304   broken clouds  8.52       1       0       0       0  ...        0   
73305   broken clouds  8.52       1       0       0       0  ...        0   
73307   broken clouds  8.52       1       0       0       0  ...        0   
...               ...   ...     ...     ...     ...     ...  ...      ...   
544725  broken clouds  8.27       0       0       0       0  ...        0   
544726  broken clouds  8.27       0       0       0       0  ...        0   
544727  broken clouds  8.27       0       0       0       0  ...        0   
544728  broken clouds  8.27       0       0       0       0  ...        0   
544729  broken clouds  8.27       0       0       0       0  ...        0   

        hour_16  hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  \
73300         0        0        0        0        0        0        0   
73302         0        0        0        0        0        0        0   
73304         0        0        0        0        0        0        0   
73305         0        0        0        0        0        0        0   
73307         0        0        0        0        0        0        0   
...         ...      ...      ...      ...      ...      ...      ...   
544725        0        0        0        0        0        0        0   
544726        0        0        0        0        0        0        0   
544727        0        0        0        0        0        0        0   
544728        0        0        0        0        0        0        0   
544729        0        0        0        0        0        0        0   

        hour_23  weekday  
73300         0        6  
73302         0        6  
73304         0        6  
73305         0        6  
73307         0        6  
...         ...      ...  
544725        1        6  
544726        1        6  
544727        1        6  
544728        1        6  
544729        1        6  

[50224 rows x 31 columns]

### Features

In [37]:
# Get all features 
A = [hour_dummies] 
B = ['description', 'temp', 'last_update']
features = (A + B)


In [38]:
print(A)

[['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']]


In [39]:
print(B)

['description', 'temp', 'last_update']


In [40]:
print(features)

[['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23'], 'description', 'temp', 'last_update']


In [41]:
stations = list(sorted(df.number.unique()))

In [42]:
print(df_monday.keys())


Index(['number', 'last_update', 'available_bikes', 'available_bike_stands',
       'description', 'temp', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weekday'],
      dtype='object')


In [43]:
y = df_monday['available_bikes']
#y = pd.DataFrame(df_monday, columns=df_monday.available_bikes)  
y = df_monday
print(y)

        number         last_update  available_bikes  available_bike_stands  \
104698      30 2021-03-22 00:00:03                0                     20   
104700      45 2021-03-22 00:00:05                0                     30   
104702      57 2021-03-22 00:00:06                4                     19   
104704     115 2021-03-22 00:00:07               30                      0   
104706      87 2021-03-22 00:00:20               11                     27   
...        ...                 ...              ...                    ...   
576008      64 2021-04-05 23:59:09                1                     39   
576009      31 2021-04-05 23:59:16               20                      0   
576010      23 2021-04-05 23:59:29                2                     28   
576011      83 2021-04-05 23:59:32               22                     17   
576012     116 2021-04-05 23:59:36               13                     17   

          description  temp  hour_0  hour_1  hour_2  hour_3  ..

In [44]:
X = pd.DataFrame(df_monday, columns=df_monday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

        number         last_update  available_bike_stands  temp  hour_0  \
104698      30 2021-03-22 00:00:03                     20  8.03       1   
104700      45 2021-03-22 00:00:05                     30  8.03       1   
104702      57 2021-03-22 00:00:06                     19  8.03       1   
104704     115 2021-03-22 00:00:07                      0  8.03       1   
104706      87 2021-03-22 00:00:20                     27  8.03       1   
...        ...                 ...                    ...   ...     ...   
576008      64 2021-04-05 23:59:09                     39 -0.15       0   
576009      31 2021-04-05 23:59:16                      0 -0.15       0   
576010      23 2021-04-05 23:59:29                     28 -0.15       0   
576011      83 2021-04-05 23:59:32                     17 -0.15       0   
576012     116 2021-04-05 23:59:36                     17 -0.15       0   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
104698       0       0    

number                     int64
last_update                int64
available_bike_stands      int64
temp                     float64
hour_0                     uint8
hour_1                     uint8
hour_2                     uint8
hour_3                     uint8
hour_4                     uint8
hour_5                     uint8
hour_6                     uint8
hour_7                     uint8
hour_8                     uint8
hour_9                     uint8
hour_10                    uint8
hour_11                    uint8
hour_12                    uint8
hour_13                    uint8
hour_14                    uint8
hour_15                    uint8
hour_16                    uint8
hour_17                    uint8
hour_18                    uint8
hour_19                    uint8
hour_20                    uint8
hour_21                    uint8
hour_22                    uint8
hour_23                    uint8
weekday                    int64
dtype: object

In [45]:
X.head(5)


number          last_update  available_bike_stands  temp  hour_0  \
104698      30  1616371203000000000                     20  8.03       1   
104700      45  1616371205000000000                     30  8.03       1   
104702      57  1616371206000000000                     19  8.03       1   
104704     115  1616371207000000000                      0  8.03       1   
104706      87  1616371220000000000                     27  8.03       1   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
104698       0       0       0       0       0  ...        0        0   
104700       0       0       0       0       0  ...        0        0   
104702       0       0       0       0       0  ...        0        0   
104704       0       0       0       0       0  ...        0        0   
104706       0       0       0       0       0  ...        0        0   

        hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  hour_23  weekday  
104698        0        0        0        0        0        0        0        0  
104700        0        0        0        0        0        0        0        0  
104702        0        0        0        0        0        0        0        0  
104704        0        0        0        0        0        0        0        0  
104706        0        0        0        0        0        0        0        0  

[5 rows x 29 columns]

### Training

In [58]:
# Taking each day individually

# Monday
i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Monday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
#     print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
#     print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)

    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
#         print(r2)
#         print('inside r2')
#         print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))
    
    

Percentage of models that have outperformed R^2 = 0.52: 73.63636363636363
81
110


In [47]:
print(df_tuesday.keys())

Index(['number', 'last_update', 'available_bikes', 'available_bike_stands',
       'description', 'temp', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weekday'],
      dtype='object')


In [48]:
y = df_tuesday['available_bikes']
print(y)

0         10
1         13
2         12
3          4
4         13
          ..
607400    16
607401    18
607402     0
607403     7
607404    24
Name: available_bikes, Length: 51999, dtype: int64


In [49]:
X = pd.DataFrame(df_tuesday, columns=df_tuesday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

        number         last_update  available_bike_stands  temp  hour_0  \
0           76 2021-03-16 19:24:43                     28   NaN       0   
1          110 2021-03-16 19:24:44                     27   NaN       0   
2           48 2021-03-16 19:24:45                     28   NaN       0   
3           79 2021-03-16 19:24:45                     23   NaN       0   
4           43 2021-03-16 19:24:52                     17   NaN       0   
...        ...                 ...                    ...   ...     ...   
607400      78 2021-04-06 23:58:52                     24  2.46       0   
607401      83 2021-04-06 23:58:53                     21  2.46       0   
607402      62 2021-04-06 23:59:20                     40  2.46       0   
607403     116 2021-04-06 23:59:20                     23  2.46       0   
607404      75 2021-04-06 23:59:25                     16  2.46       0   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
0            0       0    

number                     int64
last_update                int64
available_bike_stands      int64
temp                     float64
hour_0                     uint8
hour_1                     uint8
hour_2                     uint8
hour_3                     uint8
hour_4                     uint8
hour_5                     uint8
hour_6                     uint8
hour_7                     uint8
hour_8                     uint8
hour_9                     uint8
hour_10                    uint8
hour_11                    uint8
hour_12                    uint8
hour_13                    uint8
hour_14                    uint8
hour_15                    uint8
hour_16                    uint8
hour_17                    uint8
hour_18                    uint8
hour_19                    uint8
hour_20                    uint8
hour_21                    uint8
hour_22                    uint8
hour_23                    uint8
weekday                    int64
dtype: object

In [50]:
X.tail(5)

number          last_update  available_bike_stands  temp  hour_0  \
607400      78  1617753532000000000                     24  2.46       0   
607401      83  1617753533000000000                     21  2.46       0   
607402      62  1617753560000000000                     40  2.46       0   
607403     116  1617753560000000000                     23  2.46       0   
607404      75  1617753565000000000                     16  2.46       0   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
607400       0       0       0       0       0  ...        0        0   
607401       0       0       0       0       0  ...        0        0   
607402       0       0       0       0       0  ...        0        0   
607403       0       0       0       0       0  ...        0        0   
607404       0       0       0       0       0  ...        0        0   

        hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  hour_23  weekday  
607400        0        0        0        0        0        0        1        1  
607401        0        0        0        0        0        0        1        1  
607402        0        0        0        0        0        0        1        1  
607403        0        0        0        0        0        0        1        1  
607404        0        0        0        0        0        0        1        1  

[5 rows x 29 columns]

In [57]:
# # Tuesday

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Tuesday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
#     print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
#     print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)

    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
#         print(r2)
#         print('inside r2')
#         print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))
    







Percentage of models that have outperformed R^2 = 0.52: 74.54545454545455
82
110


In [ ]:
print(df_wednesday.keys())

In [ ]:
y = df_wednesday['available_bikes']
print(y)

In [ ]:
X = pd.DataFrame(df_wednesday, columns=df_wednesday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

In [ ]:
X.tail(5)

In [59]:
# # Wednesday 

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Wednesday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
#     print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
#     print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)

    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
#         print(r2)
#         print('inside r2')
#         print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))






Percentage of models that have outperformed R^2 = 0.52: 79.0909090909091
87
110


In [ ]:
print(df_thursday.keys())

In [ ]:
y = df_thursday['available_bikes']
print(y)

In [ ]:
X = pd.DataFrame(df_thursday, columns=df_thursday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

In [ ]:
X.tail(5)

In [60]:
# Thursday 

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Thursday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
    print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
    print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)
    print('error here: ' ,metrics.mean_squared_error(y_test, test_predictions)**0.5)
    print('r2 is ' ,r2)


    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
        print(r2)
        print('inside r2')
        print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))
      

Train predictions for station  2 :  [ 8.42475119 22.14898223  1.06080757 ...  9.20140206  7.94657004
 18.32693581]
Test predictions for station  2 :  [18.15113101 12.63868802 11.05079601 ...  4.28130956 12.45248795
 14.5660966 ]
error here:  5.539539378473503
r2 is  0.5252521670159087
0.5252521670159087
inside r2
i is  0
Train predictions for station  3 :  [20.61411199 15.16336799  9.69160321 ...  8.15397025 11.84792931
 11.82951922]
Test predictions for station  3 :  [17.04656126  9.88805476  6.71680307 ...  8.02438363  8.75863152
  4.32566196]
error here:  5.676484087553928
r2 is  0.505518428893654
0.505518428893654
inside r2
i is  1
Train predictions for station  4 :  [ 0.42989219  8.29352686  8.90163728 ... 10.03505249  5.79054496
  1.8210073 ]
Test predictions for station  4 :  [-0.7424098  10.26085781 13.24292582 ...  4.95545329  6.23438351
 10.72095307]
error here:  5.513984939511618
r2 is  0.5172329246188287
0.5172329246188287
inside r2
i is  2
Train predictions for station  5 

Test predictions for station  30 :  [22.72806654  6.05182768 15.85421561 ... 10.45512248 11.06631577
 14.18561101]
error here:  5.685551977989125
r2 is  0.4982868963129289
Train predictions for station  31 :  [ 7.53215189 18.3887124  11.87758912 ... 18.0583404  14.02104714
  5.57040285]
Test predictions for station  31 :  [ 6.36308983 19.45339695 10.32814025 ...  9.60702871 12.96194126
 18.41327528]
error here:  5.6324210361512455
r2 is  0.5017571444214242
0.5017571444214242
inside r2
i is  22
Train predictions for station  32 :  [ 3.7232503  13.87593472  3.05036609 ...  3.85709413  6.66798631
  2.30826433]
Test predictions for station  32 :  [ 6.00715177 18.47479204 11.36094412 ...  8.32934349  9.01321886
 11.87363195]
error here:  5.6098468899351275
r2 is  0.5084501271020738
0.5084501271020738
inside r2
i is  23
Train predictions for station  33 :  [ 1.35087576 20.2368089  17.64197574 ...  5.47123633 10.33386894
  5.24396452]
Test predictions for station  33 :  [ 3.00684195 12.344699

Test predictions for station  59 :  [ 2.52799544  9.35525888 17.74755028 ...  4.57561206  4.29214413
  6.25835938]
error here:  5.617005270075932
r2 is  0.5003078045042075
0.5003078045042075
inside r2
i is  42
Train predictions for station  61 :  [14.03158037  8.35043719 12.67039317 ... 18.68527361  7.70413565
 13.10963854]
Test predictions for station  61 :  [ 5.63678832  0.43416888 10.59416679 ...  5.78129934  8.48870715
 12.84466362]
error here:  5.6448421233910695
r2 is  0.49304442577322416
Train predictions for station  62 :  [10.5397657  14.61132287 13.20320984 ... 11.92650393 14.2466883
 10.58111695]
Test predictions for station  62 :  [11.19504874 17.96248486 15.95096699 ...  3.18182226  8.75436587
 12.90473366]
error here:  5.641804651587323
r2 is  0.5035749228162342
0.5035749228162342
inside r2
i is  43
Train predictions for station  63 :  [ 8.88115556  4.5256323   9.65324488 ... 11.26610663  3.44129832
  9.02131862]
Test predictions for station  63 :  [ 6.94458374  7.4103455

Test predictions for station  89 :  [10.37425986 14.27459568 16.19744223 ...  0.05283635 11.26893286
  9.65211036]
error here:  5.6010160667219395
r2 is  0.509634358298712
0.509634358298712
inside r2
i is  60
Train predictions for station  90 :  [ 8.28160998  7.07070377 14.07008686 ... 15.85681929 15.57524337
  5.37578574]
Test predictions for station  90 :  [16.98910144  1.77908902  9.78550479 ...  3.7522006   8.21660911
 14.09986249]
error here:  5.674068538938872
r2 is  0.4945259357955367
Train predictions for station  91 :  [10.43713709 11.60103167 -1.50545952 ...  8.57647384 20.37086917
  4.66604315]
Test predictions for station  91 :  [ 2.24943758  5.88412494 10.13847899 ...  7.59658422 18.99379
 19.96798121]
error here:  5.67396831186992
r2 is  0.5006573275572865
0.5006573275572865
inside r2
i is  61
Train predictions for station  92 :  [10.17114223  5.65958204 10.25228937 ...  6.32155849 14.7874251
  6.77056047]
Test predictions for station  92 :  [ 2.68146559 11.14050612 10.89

Test predictions for station  116 :  [ 8.85765739 12.78448795  3.81274334 ... 16.06896962 12.94924941
 -4.13116629]
error here:  5.595744717457863
r2 is  0.5076264612999544
0.5076264612999544
inside r2
i is  82
Train predictions for station  117 :  [18.25853309 11.22875857 14.09094683 ...  6.09712671  9.15930843
 20.97121576]
Test predictions for station  117 :  [-3.49354185 21.39542484 14.15288365 ... 13.98102214  7.69272956
  2.78196511]
error here:  5.557563366493212
r2 is  0.5153830709587006
0.5153830709587006
inside r2
i is  83
Train predictions for station  507 :  [ 2.33954105  8.45916549 18.26736453 ...  5.6443729  19.37969847
  2.84689602]
Test predictions for station  507 :  [10.40626891 13.59645073  2.88394376 ... 11.7501628   6.20469667
 10.37195673]
error here:  5.624169894241709
r2 is  0.49986093226472494
Percentage of models that have outperformed R^2 = 0.52: 76.36363636363637
84
110


In [ ]:
print(df_friday.keys())

In [ ]:
y = df_friday['available_bikes']
print(y)

In [ ]:
X = pd.DataFrame(df_friday, columns=df_friday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

In [ ]:
X.tail(5)

In [61]:
# Friday

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Friday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
    print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
    print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)
    print('error here: ' ,metrics.mean_squared_error(y_test, test_predictions)**0.5)
    print('r2 is ' ,r2)


    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
        print(r2)
        print('inside r2')
        print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))
    






Train predictions for station  2 :  [ 1.24558646 13.08836773  3.73940314 ...  9.04440703  3.96791953
  6.66022379]
Test predictions for station  2 :  [ 7.94012362 13.22208807 -0.02417914 ... 20.2680843  18.76216962
  8.98614314]
error here:  5.533129239261535
r2 is  0.518266019201203
0.518266019201203
inside r2
i is  0
Train predictions for station  3 :  [0.13887    2.17675319 5.64150915 ... 5.80788082 1.66205373 6.52125951]
Test predictions for station  3 :  [-0.27903376 13.26501497 -2.08575173 ... 17.55531962 11.18725529
  6.49562141]
error here:  5.662456152022781
r2 is  0.490885765319383
Train predictions for station  4 :  [ 7.72669324 10.71524452  4.80372983 ...  7.36150751 10.02307215
 11.55736146]
Test predictions for station  4 :  [ 8.87262237 10.0522281  -0.70886012 ... 14.82331866  1.03019033
 17.82039954]
error here:  5.619904930437718
r2 is  0.5082686588184138
0.5082686588184138
inside r2
i is  1
Train predictions for station  5 :  [ 4.68149744 13.04397253  0.73003061 ...  

error here:  5.58709200441744
r2 is  0.5087908067611124
0.5087908067611124
inside r2
i is  20
Train predictions for station  33 :  [15.59963572  3.3254978   5.66093011 ... -1.56293267  1.19397981
  5.33427288]
Test predictions for station  33 :  [ 5.31755971  5.46024542  2.91672233 ... 11.8012333  14.94873449
 10.59311601]
error here:  5.6116538243478855
r2 is  0.5117839322050715
0.5117839322050715
inside r2
i is  21
Train predictions for station  34 :  [11.04119129  8.22559151  7.37589033 ...  7.68649541 11.92930406
  9.68518874]
Test predictions for station  34 :  [ 9.10152751  3.29966456  6.97191014 ...  8.37730733 13.86443026
 11.70548691]
error here:  5.501310986118659
r2 is  0.5213863775390903
0.5213863775390903
inside r2
i is  22
Train predictions for station  36 :  [ 4.82582079  4.19207128  4.51865258 ... 18.68606515 10.57472678
 -3.7000403 ]
Test predictions for station  36 :  [ 3.97774856 18.88618346 10.42779954 ... 13.76310542 12.41412477
  6.93648098]
error here:  5.6601174

Test predictions for station  62 :  [13.03436007  8.104885   11.34092068 ... 10.10148572  1.03255995
  7.31868862]
error here:  5.615257035392619
r2 is  0.503898534999313
0.503898534999313
inside r2
i is  45
Train predictions for station  63 :  [ 8.74305057  8.42075684  0.23509732 ...  9.82182582  4.22041925
 12.69610351]
Test predictions for station  63 :  [26.01554782  5.50171351 -1.11411841 ... -0.39394612 13.75553887
  9.27704924]
error here:  5.649901578703544
r2 is  0.5022516283809085
0.5022516283809085
inside r2
i is  46
Train predictions for station  64 :  [23.58688076  4.52561287  0.37438192 ...  8.49722003  6.005744
  8.48994794]
Test predictions for station  64 :  [43.23994479 10.96055554 15.40043047 ...  8.88024307  8.32469866
 14.50469512]
error here:  5.602805442895192
r2 is  0.5018827885530839
0.5018827885530839
inside r2
i is  47
Train predictions for station  65 :  [ 2.56364061  8.85795041 10.06905935 ... 12.90035313 10.37426725
  5.71170628]
Test predictions for stati

Test predictions for station  91 :  [-2.38508113  5.37598086 12.86210429 ... 12.25825971 13.92908829
  3.63153379]
error here:  5.721619868931902
r2 is  0.4955958129859652
Train predictions for station  92 :  [ 2.25541672  2.0516105   7.50381914 ...  6.56464329 10.77429529
 10.37516819]
Test predictions for station  92 :  [12.39771163  8.9379693   9.20868239 ...  9.16136048  9.80788021
  8.98502925]
error here:  5.595930409118681
r2 is  0.5172772856098715
0.5172772856098715
inside r2
i is  64
Train predictions for station  93 :  [21.42131467 11.79431865  9.97345884 ... -0.74296296  2.05283438
  5.3267855 ]
Test predictions for station  93 :  [ 9.05975613 12.50281138  7.93971457 ...  7.13638829  9.25844932
  4.64539045]
error here:  5.655818854981455
r2 is  0.5098469516954907
0.5098469516954907
inside r2
i is  65
Train predictions for station  94 :  [18.8555675   5.13869282  8.50562553 ... 15.37310814  2.55092189
  3.02625096]
Test predictions for station  94 :  [-4.935919    9.02137972

Test predictions for station  117 :  [ 9.94741005  9.69348425 16.43136873 ...  3.74952873  9.87509379
  4.28968963]
error here:  5.645915268917505
r2 is  0.4937049400284299
Train predictions for station  507 :  [11.62048165 11.02936025 10.20139442 ...  3.85212734  7.0212125
 -0.89983742]
Test predictions for station  507 :  [14.73371885  4.10157561  0.85055293 ...  6.76736443  1.46428722
 12.00371044]
error here:  5.540573182164572
r2 is  0.5128641003527439
0.5128641003527439
inside r2
i is  83
Percentage of models that have outperformed R^2 = 0.52: 76.36363636363637
84
110


In [ ]:
print(df_saturday.keys())

In [ ]:
y = df_saturday['available_bikes']
print(y)

In [ ]:
X = pd.DataFrame(df_saturday, columns=df_saturday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

In [ ]:
X.tail(5)

In [62]:
# Saturday

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Saturday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
    print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
    print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)
    print('error here: ' ,metrics.mean_squared_error(y_test, test_predictions)**0.5)
    print('r2 is ' ,r2)


    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
        print(r2)
        print('inside r2')
        print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))

Train predictions for station  2 :  [12.91952396  3.48306598 12.82600158 ...  5.3917644   3.97446564
  9.00117424]
Test predictions for station  2 :  [ 9.9479762  12.66362691 12.18938386 ... 11.98890293 12.00929314
 12.86894732]
error here:  5.633846526315639
r2 is  0.5086599361407744
0.5086599361407744
inside r2
i is  0
Train predictions for station  3 :  [12.86891022  8.66300799  0.3331162  ...  7.5296038   8.15193884
  8.50939965]
Test predictions for station  3 :  [ 4.77274166  2.46843731  4.32732535 ...  5.09783551  4.8821818
 14.3313661 ]
error here:  5.659445981801698
r2 is  0.5039654103323914
0.5039654103323914
inside r2
i is  1
Train predictions for station  4 :  [ 9.43939798  4.0380864  15.83152542 ...  2.90578142  1.82439811
  8.73624209]
Test predictions for station  4 :  [18.44532265  5.61438706 17.89144982 ...  7.71635568  2.22673069
  4.59991752]
error here:  5.701381580261964
r2 is  0.49439045303904694
Train predictions for station  5 :  [10.4429142   0.70458039 16.1500

Train predictions for station  30 :  [15.94621593 11.58308496  0.88258298 ...  5.20878817  3.99223094
 10.79560604]
Test predictions for station  30 :  [ 9.10009696 19.65792926 21.6132867  ... 17.60766227 11.65496483
 14.74973892]
error here:  5.664604383497402
r2 is  0.5016506155214409
0.5016506155214409
inside r2
i is  21
Train predictions for station  31 :  [19.19595121 19.46961274 11.20168677 ...  8.26381354  4.99778354
  5.85424788]
Test predictions for station  31 :  [10.81448373 -0.27412476  8.95409472 ... 15.56356532  8.99837396
 14.47164468]
error here:  5.621696678144451
r2 is  0.5054992725086319
0.5054992725086319
inside r2
i is  22
Train predictions for station  32 :  [ 7.61007159 11.60642376  5.33158162 ... 11.80576326 42.90030022
 12.87760167]
Test predictions for station  32 :  [16.96539082 14.42421085  3.09327052 ...  1.28711166 13.09579052
  8.46883678]
error here:  5.549231877951848
r2 is  0.5034466356170668
0.5034466356170668
inside r2
i is  23
Train predictions for 

Test predictions for station  58 :  [ 2.63593024 14.83714836 10.41430082 ...  9.81907357 10.73599787
 15.76387717]
error here:  5.574051894339702
r2 is  0.5119390918768902
0.5119390918768902
inside r2
i is  42
Train predictions for station  59 :  [ 2.966783   12.24212514 16.36637527 ...  6.72100941 12.38176577
  2.14539561]
Test predictions for station  59 :  [ 7.49659761  2.87837121 17.79433838 ...  6.07394221 17.47171784
 15.38846706]
error here:  5.677037341845439
r2 is  0.4870528507432639
Train predictions for station  61 :  [ 9.20185125 15.11755872 14.77505216 ...  4.55495838  0.78803989
 13.86632375]
Test predictions for station  61 :  [10.80352932  7.64867243 10.02910447 ... 10.02958565  2.02545379
  8.09766235]
error here:  5.567772411185935
r2 is  0.5174478632280819
0.5174478632280819
inside r2
i is  43
Train predictions for station  62 :  [ 6.75766342  6.20171041  9.0655766  ... 12.123698   14.16959141
 13.35244434]
Test predictions for station  62 :  [ 8.58414058  6.91310781

Test predictions for station  88 :  [10.51368394  6.94388118 10.97362633 ... 13.01050695 -0.86637931
 -0.73262274]
error here:  5.613293274200235
r2 is  0.5078742336291385
0.5078742336291385
inside r2
i is  58
Train predictions for station  89 :  [ 9.30925267 16.53714813 17.1732504  ...  8.48216694  6.47370439
  6.55589833]
Test predictions for station  89 :  [-1.37887786  9.70657325  6.47211201 ...  3.0587667   7.13550972
  4.759498  ]
error here:  5.572862453993736
r2 is  0.5086096990195329
0.5086096990195329
inside r2
i is  59
Train predictions for station  90 :  [ 1.7522804   5.74565573 11.32548097 ... 21.03001883  1.73150235
  0.90725952]
Test predictions for station  90 :  [18.03444675 12.58320314  7.24489946 ...  8.7117377   0.62843838
  6.77613953]
error here:  5.599967589293529
r2 is  0.5033916918759596
0.5033916918759596
inside r2
i is  60
Train predictions for station  91 :  [10.16263818 10.39804161 -0.41316613 ...  8.55819522 11.7314962
 17.43682599]
Test predictions for st

Test predictions for station  114 :  [ 6.65209429 -0.80629989  3.36688479 ... 11.12719818  9.90364662
 12.01254574]
error here:  5.6093638326573085
r2 is  0.5106835707835793
0.5106835707835793
inside r2
i is  78
Train predictions for station  115 :  [ 5.62500308 20.38825948  0.74062336 ... 13.33335156  4.60375892
  7.78581807]
Test predictions for station  115 :  [-0.75037694  9.07424589  3.21117727 ... 10.11328766 18.91548571
  8.5637972 ]
error here:  5.59611780428625
r2 is  0.5149554131287144
0.5149554131287144
inside r2
i is  79
Train predictions for station  116 :  [ 8.59906711  6.04661263  6.54457485 ... 14.53134549  7.84983531
  8.15551298]
Test predictions for station  116 :  [ 6.32600743  9.51052915 14.14194993 ...  2.05692514 18.68770183
 12.17236078]
error here:  5.562205225987037
r2 is  0.5156256930172041
0.5156256930172041
inside r2
i is  80
Train predictions for station  117 :  [22.14631172  8.1609407   8.21626129 ...  8.40230207 12.33828797
  6.21464492]
Test predictions

In [52]:
print(df_sunday.keys())

Index(['number', 'last_update', 'available_bikes', 'available_bike_stands',
       'description', 'temp', 'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4',
       'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
       'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
       'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weekday'],
      dtype='object')


In [53]:
y = df_sunday['available_bikes']
print(y)

73300      9
73302     20
73304     21
73305      6
73307      6
          ..
544725    37
544726     9
544727     9
544728     2
544729     8
Name: available_bikes, Length: 50224, dtype: int64


In [54]:
X = pd.DataFrame(df_sunday, columns=df_sunday.keys()) 
del X['available_bikes']
del X['description']
print(X)

X['last_update'] = X['last_update'].astype('int64')

X.dtypes

        number         last_update  available_bike_stands  temp  hour_0  \
73300       36 2021-03-21 00:00:00                     31  8.52       1   
73302        3 2021-03-21 00:00:04                      0  8.52       1   
73304       92 2021-03-21 00:00:09                     19  8.52       1   
73305       72 2021-03-21 00:00:09                     25  8.52       1   
73307       82 2021-03-21 00:00:15                     16  8.52       1   
...        ...                 ...                    ...   ...     ...   
544725      69 2021-04-04 23:59:26                      2  8.27       0   
544726      23 2021-04-04 23:59:26                     21  8.27       0   
544727      73 2021-04-04 23:59:27                     21  8.27       0   
544728       3 2021-04-04 23:59:35                     18  8.27       0   
544729      99 2021-04-04 23:59:36                     22  8.27       0   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
73300        0       0    

number                     int64
last_update                int64
available_bike_stands      int64
temp                     float64
hour_0                     uint8
hour_1                     uint8
hour_2                     uint8
hour_3                     uint8
hour_4                     uint8
hour_5                     uint8
hour_6                     uint8
hour_7                     uint8
hour_8                     uint8
hour_9                     uint8
hour_10                    uint8
hour_11                    uint8
hour_12                    uint8
hour_13                    uint8
hour_14                    uint8
hour_15                    uint8
hour_16                    uint8
hour_17                    uint8
hour_18                    uint8
hour_19                    uint8
hour_20                    uint8
hour_21                    uint8
hour_22                    uint8
hour_23                    uint8
weekday                    int64
dtype: object

In [55]:
X.tail(5)

number          last_update  available_bike_stands  temp  hour_0  \
544725      69  1617580766000000000                      2  8.27       0   
544726      23  1617580766000000000                     21  8.27       0   
544727      73  1617580767000000000                     21  8.27       0   
544728       3  1617580775000000000                     18  8.27       0   
544729      99  1617580776000000000                     22  8.27       0   

        hour_1  hour_2  hour_3  hour_4  hour_5  ...  hour_15  hour_16  \
544725       0       0       0       0       0  ...        0        0   
544726       0       0       0       0       0  ...        0        0   
544727       0       0       0       0       0  ...        0        0   
544728       0       0       0       0       0  ...        0        0   
544729       0       0       0       0       0  ...        0        0   

        hour_17  hour_18  hour_19  hour_20  hour_21  hour_22  hour_23  weekday  
544725        0        0        0        0        0        0        1        6  
544726        0        0        0        0        0        0        1        6  
544727        0        0        0        0        0        0        1        6  
544728        0        0        0        0        0        0        1        6  
544729        0        0        0        0        0        0        1        6  

[5 rows x 29 columns]

In [56]:
# Sunday

i = 0

for station in stations:
    
    
    # This is from https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    model = LinearRegression().fit(X_train, y_train)

    file_name='Sunday-'
    # Write to a pickle file
    #with open('/ddbikes/pickles/Monday-' + str(station) + '.pkl', 'wb') as handle:
    with open(file_name + str(station) + '.pkl', 'wb') as handle:
        pickle.dump(model, handle, protocol = pickle.HIGHEST_PROTOCOL)

    # Create vectors of predictions
    train_predictions = model.predict(X_train)
    print('Train predictions for station ', station, ": ", train_predictions)

    test_predictions = model.predict(X_test)
    print('Test predictions for station ', station, ": ", test_predictions)


    # R^2 - R-squared is a statistical measure of how close the data are to the fitted regression line.
    # r2 = metrics.r2_score(y_true, y_predict)
    r2 = metrics.r2_score(y_test, test_predictions)
    print('error here: ' ,metrics.mean_squared_error(y_test, test_predictions)**0.5)
    print('r2 is ' ,r2)


    # how well does the model behave compared to a chosen r2 ? 
    # an r2 of 0.75 indicates that 25% of the variability in the outcome data cannot be explained by the model
    # initialise to get how many times the model outperforms r2 = 0.75
    if r2 > 0.5:
        print(r2)
        print('inside r2')
        print('i is ', i)
        i +=1

print('Percentage of models that have outperformed R^2 = 0.52:', (i /110) * 100)  # there are 110 stations
print(i)
print(len(stations))
    



Train predictions for station  2 :  [10.62047496  7.96050195  8.88300665 ... 17.86876208 11.79680495
  0.51218863]
Test predictions for station  2 :  [14.91696513  3.1096487  10.4670779  ... 11.4843686  19.31985732
  9.53581001]
error here:  5.532871234810343
r2 is  0.5187334632519416
0.5187334632519416
inside r2
i is  0
Train predictions for station  3 :  [ 5.39219491  0.35197211 12.80533685 ... 10.49107951 11.2237431
  5.56459621]
Test predictions for station  3 :  [13.93411797 12.39142533  5.53295758 ...  3.20024124 11.13136751
 10.61666929]
error here:  5.546934284744106
r2 is  0.5185917461795937
0.5185917461795937
inside r2
i is  1
Train predictions for station  4 :  [ 9.67804705 10.60092981 10.25868793 ... 15.71951389  7.16241948
  7.40713311]
Test predictions for station  4 :  [8.16899749 0.91762171 5.64436567 ... 4.67558472 8.15863969 1.77564666]
error here:  5.6423832491002885
r2 is  0.5094765941394259
0.5094765941394259
inside r2
i is  2
Train predictions for station  5 :  [ 

Train predictions for station  31 :  [ 5.83444539 11.13004523  1.40156895 ...  4.0924907  10.89553717
  4.34133544]
Test predictions for station  31 :  [12.49215512  4.31426206 21.03963294 ... 11.09586558  6.86649459
 15.28693055]
error here:  5.595826305569169
r2 is  0.5118018340252437
0.5118018340252437
inside r2
i is  24
Train predictions for station  32 :  [15.82581011 14.16218544  9.70193816 ... 12.65562792  3.38166606
  8.18719489]
Test predictions for station  32 :  [ 0.70717316  8.51984858  3.42549466 ... 11.685335    9.54373249
  5.85031052]
error here:  5.604006358355227
r2 is  0.5103004388798638
0.5103004388798638
inside r2
i is  25
Train predictions for station  33 :  [15.4662657   8.99981316  9.83070449 ...  7.19589885 18.87726636
  7.99233986]
Test predictions for station  33 :  [16.07859283 10.55609633 11.22409668 ... 14.36432996  6.4305572
 -0.40318704]
error here:  5.631656507498373
r2 is  0.5070173194676033
0.5070173194676033
inside r2
i is  26
Train predictions for s

Test predictions for station  61 :  [12.44031315  9.38453439  3.19396326 ...  3.42917629 -2.29569416
 12.47374909]
error here:  5.56028864900688
r2 is  0.5110680760187003
0.5110680760187003
inside r2
i is  46
Train predictions for station  62 :  [ 8.0201342  11.56480451  1.47770109 ...  7.57112604  9.98462091
 11.11336998]
Test predictions for station  62 :  [ 4.49950566  2.90848215 12.15594907 ...  5.78807161 22.05043504
  8.26050592]
error here:  5.5789669202030225
r2 is  0.5117320797541196
0.5117320797541196
inside r2
i is  47
Train predictions for station  63 :  [ 5.58925933  0.84726535 11.69788766 ...  2.85188069 10.33727072
  4.83973712]
Test predictions for station  63 :  [ 6.22053014  1.72235925 11.49942314 ... 11.57523417 16.2379072
  1.86308155]
error here:  5.572461259465233
r2 is  0.5105128851450715
0.5105128851450715
inside r2
i is  48
Train predictions for station  64 :  [17.83784172 11.25022101 11.48536565 ...  0.70130709  3.93357673
 16.30807371]
Test predictions for st

Test predictions for station  94 :  [ 5.68737561  5.37718523 11.13547361 ... 11.46977483 10.59494431
  5.69576168]
error here:  5.680268701575919
r2 is  0.4883722636621397
Train predictions for station  95 :  [ 6.07806296 13.37080045  3.5882329  ... 16.37072601 12.39908988
  8.02074918]
Test predictions for station  95 :  [11.69850939 20.02248127 10.23849961 ... -1.08042909 11.73293501
  3.24038173]
error here:  5.665430871081946
r2 is  0.5089988946629858
0.5089988946629858
inside r2
i is  66
Train predictions for station  96 :  [ 7.07482017  7.51646938 22.90109502 ...  3.54179746  5.00312072
 19.65211493]
Test predictions for station  96 :  [17.48593673 10.33190622 15.87315686 ...  6.77992836  7.50960971
  3.3563747 ]
error here:  5.698787918032977
r2 is  0.4931611755695575
Train predictions for station  97 :  [ 8.25157966 15.57111453  0.47120455 ...  3.24778169  0.39819147
 -0.73508857]
Test predictions for station  97 :  [ 4.69933781  9.99201428  6.49537523 ...  7.77640569 12.705961